In [13]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


# df = pd.read_csv('../films_db2.csv')
# df.head(10)

In [15]:
from db.database_mysql import engine

df = pd.read_sql_query('''SELECT jp.title, jp.`year`, jp.director, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, jp.total_spectator, jp.copies, fa1.rating_press, fa1.rating_public, fa1.casting, fa1.budget, fa1.lang, fa1.visa, fa1.award 
FROM films_jp as jp
LEFT JOIN films_allo fa1 ON fa1.id_jp = jp.id                     
  where fa1.year_allo is not null and fa1.year_allo != -1
  group by fa1.id_jp, jp.`year`, jp.director''', engine)

df.head()

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997,claude chabrol,france,6300,thriller,-1,172230,-1,3,458125,234,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999,patrice leconte,france,5400,drame,-1,154881,-1,5,663390,198,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",990000000,"[""francais""]",90876,1
2,grosse fatigue,1994,michel blanc,france,5220,comedie,-1,417021,-1,2,2015230,189,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1
3,braveheart,1995,mel gibson,etatsunis,9900,aventure action,-1,355642,-1,3,1231534,302,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",72000000,"[""anglais""]",88091,9
4,regarde les hommes tomber,1994,jacques audiard,france,6600,thriller,-1,39739,-1,10,218831,46,-1.0,3.3,"[""Jean-Louis Trintignant"", ""Mathieu Kassovitz""...",-1,"[""francais""]",84098,3


In [ ]:
# from db.database_mysql import engine

# df = pd.read_sql_query('''SELECT jp.title, jp.`year`, jp.director, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, jp.total_spectator, jp.copies, fa1.rating_press, fa1.rating_public, fa1.casting, fa1.budget, fa1.lang, fa1.visa, fa1.award 
# FROM films_jp as jp
# LEFT JOIN films_allo fa1 ON fa1.id_jp = jp.id 
#                         and fa1.year_allo = jp.`year` ''', engine)
# # and fa1.director_allo like CONCAT("%", jp.director, "%")

# df.head()

In [16]:
df

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997,claude chabrol,france,6300,thriller,-1,172230,-1,3,458125,234,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999,patrice leconte,france,5400,drame,-1,154881,-1,5,663390,198,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",990000000,"[""francais""]",90876,1
2,grosse fatigue,1994,michel blanc,france,5220,comedie,-1,417021,-1,2,2015230,189,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1
3,braveheart,1995,mel gibson,etatsunis,9900,aventure action,-1,355642,-1,3,1231534,302,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",72000000,"[""anglais""]",88091,9
4,regarde les hommes tomber,1994,jacques audiard,france,6600,thriller,-1,39739,-1,10,218831,46,-1.0,3.3,"[""Jean-Louis Trintignant"", ""Mathieu Kassovitz""...",-1,"[""francais""]",84098,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,be happy,2008,mike leigh,grandebretagne,7080,comedie,18126,110442,90372,9,362884,128,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",-1,"[""anglais""]",121057,1
2203,m butterfly,1993,david cronenberg,etatsunis,6060,drame,-1,32351,-1,10,101680,44,-1.0,3.6,"[""Jeremy Irons"", ""John Lone"", ""Barbara Sukowa""...",-1,"[""anglais""]",-1,0
2204,petits freres,1999,jacques doillon,france,5520,drame,-1,51749,-1,14,176880,-1,3.9,2.7,"[""Stephanie Touly"", ""Iliès Sefraoui"", ""Mustaph...",-1,"[""francais""]",-1,0
2205,cookies fortune,1999,robert altman,etatsunis,7080,comedie,-1,63249,-1,13,239473,-1,3.8,3.3,"[""Glenn Close"", ""Julianne Moore"", ""Liv Tyler"",...",-1,"[""anglais""]",97081,0


In [17]:
# df = df.drop(["title","visa"],axis=1)
# df.head(15)

In [18]:
df.nunique()

title              2206
year                 34
director            358
country              21
duration            118
genre                19
first_day          1695
first_week         2205
first_weekend      1566
hebdo_rank           20
total_spectator    2204
copies              739
rating_press         41
rating_public        39
casting            2096
budget              300
lang                175
visa               1781
award                27
dtype: int64

In [19]:
df.isnull().sum()

title              0
year               0
director           0
country            0
duration           0
genre              0
first_day          0
first_week         0
first_weekend      0
hebdo_rank         0
total_spectator    0
copies             0
rating_press       0
rating_public      0
casting            0
budget             0
lang               0
visa               0
award              0
dtype: int64

In [20]:
df.loc[df["rating_press"].isnull()]

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award


In [21]:
colonnes_numeriques = ["first_day","first_weekend",'budget']

df[colonnes_numeriques] = df[colonnes_numeriques].replace(-1, np.nan)

imputer = KNNImputer(n_neighbors=5)

df_imputed = imputer.fit_transform(df[colonnes_numeriques])

df[colonnes_numeriques] = df_imputed

df

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997,claude chabrol,france,6300,thriller,74284.631579,172230,368944.741401,3,458125,234,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",1.549450e+08,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999,patrice leconte,france,5400,drame,75572.200000,154881,412897.600000,5,663390,198,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",9.900000e+08,"[""francais""]",90876,1
2,grosse fatigue,1994,michel blanc,france,5220,comedie,74284.631579,417021,368944.741401,2,2015230,189,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",1.549450e+08,"[""francais""]",82266,1
3,braveheart,1995,mel gibson,etatsunis,9900,aventure action,45088.600000,355642,206923.400000,3,1231534,302,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",7.200000e+07,"[""anglais""]",88091,9
4,regarde les hommes tomber,1994,jacques audiard,france,6600,thriller,74284.631579,39739,368944.741401,10,218831,46,-1.0,3.3,"[""Jean-Louis Trintignant"", ""Mathieu Kassovitz""...",1.549450e+08,"[""francais""]",84098,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,be happy,2008,mike leigh,grandebretagne,7080,comedie,18126.000000,110442,90372.000000,9,362884,128,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",3.072000e+07,"[""anglais""]",121057,1
2203,m butterfly,1993,david cronenberg,etatsunis,6060,drame,74284.631579,32351,368944.741401,10,101680,44,-1.0,3.6,"[""Jeremy Irons"", ""John Lone"", ""Barbara Sukowa""...",1.549450e+08,"[""anglais""]",-1,0
2204,petits freres,1999,jacques doillon,france,5520,drame,74284.631579,51749,368944.741401,14,176880,-1,3.9,2.7,"[""Stephanie Touly"", ""Iliès Sefraoui"", ""Mustaph...",1.549450e+08,"[""francais""]",-1,0
2205,cookies fortune,1999,robert altman,etatsunis,7080,comedie,74284.631579,63249,368944.741401,13,239473,-1,3.8,3.3,"[""Glenn Close"", ""Julianne Moore"", ""Liv Tyler"",...",1.549450e+08,"[""anglais""]",97081,0


In [22]:
df.isnull().sum()

title              0
year               0
director           0
country            0
duration           0
genre              0
first_day          0
first_week         0
first_weekend      0
hebdo_rank         0
total_spectator    0
copies             0
rating_press       0
rating_public      0
casting            0
budget             0
lang               0
visa               0
award              0
dtype: int64

In [23]:
df.nunique()

title              2206
year                 34
director            358
country              21
duration            118
genre                19
first_day          1823
first_week         2205
first_weekend      1822
hebdo_rank           20
total_spectator    2204
copies              739
rating_press         41
rating_public        39
casting            2096
budget              780
lang                175
visa               1781
award                27
dtype: int64

In [24]:
df.to_csv('Dataset_analyse.csv', index=False)
df

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997,claude chabrol,france,6300,thriller,74284.631579,172230,368944.741401,3,458125,234,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",1.549450e+08,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999,patrice leconte,france,5400,drame,75572.200000,154881,412897.600000,5,663390,198,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",9.900000e+08,"[""francais""]",90876,1
2,grosse fatigue,1994,michel blanc,france,5220,comedie,74284.631579,417021,368944.741401,2,2015230,189,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",1.549450e+08,"[""francais""]",82266,1
3,braveheart,1995,mel gibson,etatsunis,9900,aventure action,45088.600000,355642,206923.400000,3,1231534,302,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",7.200000e+07,"[""anglais""]",88091,9
4,regarde les hommes tomber,1994,jacques audiard,france,6600,thriller,74284.631579,39739,368944.741401,10,218831,46,-1.0,3.3,"[""Jean-Louis Trintignant"", ""Mathieu Kassovitz""...",1.549450e+08,"[""francais""]",84098,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,be happy,2008,mike leigh,grandebretagne,7080,comedie,18126.000000,110442,90372.000000,9,362884,128,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",3.072000e+07,"[""anglais""]",121057,1
2203,m butterfly,1993,david cronenberg,etatsunis,6060,drame,74284.631579,32351,368944.741401,10,101680,44,-1.0,3.6,"[""Jeremy Irons"", ""John Lone"", ""Barbara Sukowa""...",1.549450e+08,"[""anglais""]",-1,0
2204,petits freres,1999,jacques doillon,france,5520,drame,74284.631579,51749,368944.741401,14,176880,-1,3.9,2.7,"[""Stephanie Touly"", ""Iliès Sefraoui"", ""Mustaph...",1.549450e+08,"[""francais""]",-1,0
2205,cookies fortune,1999,robert altman,etatsunis,7080,comedie,74284.631579,63249,368944.741401,13,239473,-1,3.8,3.3,"[""Glenn Close"", ""Julianne Moore"", ""Liv Tyler"",...",1.549450e+08,"[""anglais""]",97081,0
